<center>

## MAGICIAN: Mining and gAmblinG servIces sCrapIng and ANalysis

Matteo Martini 636694

In [111]:
mining_pool_url = "https://en.bitcoin.it/wiki/DeepBit9"
full_wallet_address = "https://www.walletexplorer.com/wallet/"


#Individuare nel DataSet tutte le transazioni relative a Deepbit.net,ovvero le transazioni che presentano almeno un indirizzo di input o di output appartenente a Deepbit_addrs.
#servizio scommesse = DiceOnCrack

## Setup Ambiente

In [75]:
# Librerie concesse
import pandas as pd

# Per scraping (scegline uno o entrambi, in base alle esigenze delle pagine)
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Per formati efficienti (PyArrow)
import pyarrow as pa
import pyarrow.parquet as pq

# Utility
from pathlib import Path
from datetime import datetime

import numpy as np


from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
import time
import requests
import re
from bs4 import BeautifulSoup

# Percorsi (aggiorna con il tuo percorso locale)
DATA_DIR = Path('./Datasets')   # metti qui i CSV scaricati

PARQUET_DIR = Path('./data_parquet')
PARQUET_DIR.mkdir(parents=True, exist_ok=True)
print('Environment ready.')

Environment ready.


## 1) Costruzione dei Dataframe dal Dataset

## Directory Datasets


In [76]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

## Definizone Data Type
Sapendo cosa contengono i file csv, possiamo forzare la conversione in determiinati tipi per ridurre l'impatto in memoria dei dataset

In [83]:
dtype_transactions = {
    "timestamp": np.int32,          # prima colonna: secondi (1231469xxx) rientra in int32
    "tx_id": np.int32,              # seconda colonna: contatore 1…10 572 827
    "block_height": np.int32,       # terza colonna: 1…214 562
    "prev_tx_id": np.int32,         # quarta colonna: seguono il tx_id precedente
    "has_witness": np.uint8,        # quinta colonna 0/1
    "fee": np.int32,                # sesta colonna: 0…100 000 (int32 più che sufficiente)
}


dtype_inputs = {
    "input_id": np.int32,
    "tx_id": np.int32,
    "prev_output_id": np.int32,
    "input_index": np.uint8,
}


dtype_outputs = {
    "col0":np.int32,
    "col1":np.int32,
    "col2":np.int8,
    "col3":np.int32,
    "col4":np.int64,
    "col5":"uint8"
}

dtype_mapping = {
    "address": "category",   # stringhe ripetute → category riduce drasticamente
    "address_id": "Int32",   # valori 0–8 708 820 entrano in 32 bit
}


columns_inputs = [
    "tx_id",
    "prev_tx_id",
    "prev_tx_pos"
]
columns_transactions = [
    "timestamp", "tx_id", "block_height",
     "has_witness", "fee"
]
columns_outputs = [
    "tx_id","position",
    "addressId","amount","script_type"
]
columns_mapping = [
    "hash", "addressId"
]

## Caricamento dei Dataset

In [84]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

transactions = pd.read_csv(transactions_csv,header=None,names=columns_transactions,dtype=dtype_transactions)
inputs_df    = pd.read_csv(inputs_csv,header=None,names=columns_inputs,dtype=dtype_inputs)
outputs_df   = pd.read_csv(outputs_csv,header=None,names=columns_outputs,dtype=dtype_outputs)
mapping_df   = pd.read_csv(mapping_csv,header=None,names=columns_mapping,dtype=dtype_mapping)

## Operazioni su Dataset

In [ ]:
display(inputs_df)
display(outputs_df)
display(mapping_df)


display(transactions)
merged_outputs = outputs_df.merge(mapping_df,on="addressId",how="left")

,tx_id,prev_tx_id,prev_tx_pos
0,171,9,0
1,183,171,1
2,185,183,1
3,187,185,1
4,192,187,0
...,...,...,...
21378765,10572823,10572820,7
21378766,10572824,10572820,5
21378767,10572825,10572820,6
21378768,10572826,10572820,2


,tx_id,position,addressId,amount,script_type
0,0,0,0,5000000000,1
1,1,0,1,5000000000,1
2,2,0,2,5000000000,1
3,3,0,3,5000000000,1
4,4,0,4,5000000000,1
...,...,...,...,...,...
24613794,10572825,0,8707915,1,2
24613795,10572825,1,6137715,9899999,2
24613796,10572826,0,8707915,26651306,2
24613797,10572826,1,6137711,7991620447,2


,hash,addressId
0,1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa,0
1,12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX,1
2,1HLoD9E4SDFFPDiYfNYnkBLQ85Y51J3Zb1,2
3,1FvzCLoTPGANNjWoUo6jUGuAG3wg1w4YjR,3
4,15ubicBBWFnvoZLT7GiU2qxjRaKJPdkDMG,4
...,...,...
8708816,13dJEWNK55L5Zbh8ZVqpvmpritkEYHD1nE,8708816
8708817,1GA1vXcj4UMwcgr7nBftkpxXE6RrBwwm2M,8708817
8708818,1DrJbVsNJDweTQgKAwrBNyKuvtFBS6iKNK,8708818
8708819,1Mtcwd8jsoMYdwBxcybBkFWm1NNCRMRToh,8708819


,timestamp,tx_id,block_height,has_witness,fee
0,1231006505,0,0,1,0
1,1231469665,1,1,1,0
2,1231469744,2,2,1,0
3,1231470173,3,3,1,0
4,1231470988,4,4,1,0
...,...,...,...,...,...
10572824,1356997957,214562,10572822,0,100000
10572825,1356997957,214562,10572823,0,100000
10572826,1356997957,214562,10572824,0,100000
10572827,1356997957,214562,10572825,0,100000


In [87]:
display(merged_outputs)

,tx_id,position,addressId,amount,script_type,hash
0,0,0,0,5000000000,1,1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa
1,1,0,1,5000000000,1,12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX
2,2,0,2,5000000000,1,1HLoD9E4SDFFPDiYfNYnkBLQ85Y51J3Zb1
3,3,0,3,5000000000,1,1FvzCLoTPGANNjWoUo6jUGuAG3wg1w4YjR
4,4,0,4,5000000000,1,15ubicBBWFnvoZLT7GiU2qxjRaKJPdkDMG
...,...,...,...,...,...,...
24613794,10572825,0,8707915,1,2,1DYve98WqJQmx6cDdW2EQFpnqfwptjV4e9
24613795,10572825,1,6137715,9899999,2,1AdN2my8NxvGcisPGYeQTAKdWJuUzNkQxG
24613796,10572826,0,8707915,26651306,2,1DYve98WqJQmx6cDdW2EQFpnqfwptjV4e9
24613797,10572826,1,6137711,7991620447,2,1L4EThM6x3Rd2PjNbs1U136FpMq4Gmo3fJ


## 2) Recupero informazioni da WalletExplorer


### Definizione Funzioni Helper per Scraping

In [9]:
wallet_explorer_url = "https://www.walletexplorer.com/"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/127.0.0.1 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.walletexplorer.com/",
}

In [135]:
def next_page(driver):
    #cerco il link alla prossima pagina
    for _ in ("Next","Last"):
        try:
            el = driver.find_element(By.XPATH, f'//div[@class="paging"]//a[contains(., "{_}")]')
            url = el.get_attribute("href")
            el.click()
            return url
        except:
            return None
        
def find_next_link(driver,element):
    try:
        el = driver.find_element(By.XPATH, f'//span[@class="showother"]//a[contains(., "{element}")]')
        url = el.get_attribute("href")
        el.click()
        return url
    except:
        return None

# def get_transactions_link(soup,class_):
#     transactions = soup.find_all('tr',class_=class_)
#     page_links = []
#     for trans in transactions:
#         txid = trans.find('td',class_="txid").find('a')['href']
#         page_links.append(txid)
#     return page_links

# def safe_get(session, url, tries=3, delay=1):
    for attempt in range(tries):
        try:
            return session.get(url, timeout=10)
        except (requests.exceptions.ConnectionError,
                requests.exceptions.RemoteDisconnectedError):
            if attempt == tries - 1:
                raise
            time.sleep(delay * (attempt + 1))  # 1s, poi 2s, poi 3s…

def scrape_step(driver):
    #recupero le entries
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # seleziona tutti gli elementi che hanno l’attributo data-wallet-id
    wallet_elements = soup.select("td a[href^='/address/']")
    print(wallet_elements)  
    # estrai il valore dell’attributo
    return [re.search(r"/address/([^/?#]+)", w.get("href")).group(1) for w in wallet_elements if w.get("href") and re.search(r"/address/([^/?#]+)", w.get("href"))]




### Definizione Funzione per Scraping

In [132]:
def scrape_wallet_explorer(wallet_name):
    driver = webdriver.Chrome()
    driver.get(wallet_explorer_url)
    
    #ricerca del wallet tramite form
    driver.find_element(By.NAME,'q').send_keys(wallet_name)
    driver.find_element(By.TAG_NAME,'form').submit()
    
    
    #mettere funzione di selenium
    time.sleep(1)
    
    #mostra wallet
    tri = find_next_link(driver,"show wallet addresses")
    if tri == None: return
    #mettere funzione di selenium
    time.sleep(1)

    try:
        while True:
            #recupero di tutti gli input/output della pagina
            yield from scrape_step(driver)

            if next_page(driver) == None:break;
    except:
            print('Something went wrong')
    finally:
        driver.close()    
   
#https://www.walletexplorer.com/wallet/


### Probabilmente non serve ma si cancella solo a progetto finito

In [ ]:
# BLOCK_RE = re.compile(r"\d+")
# FEE_RE = re.compile(r"\d+(?:\.\d+)?")

# def get_info_from_transaction_table(trans_table_link,session):
#     # res = session.get(f'{"https://www.walletexplorer.com"}{trans_table_link}',timeout=10)
#     res = safe_get(session,f'{"https://www.walletexplorer.com"}{trans_table_link}')
#     soup = BeautifulSoup(res.text,"html.parser")
#     cells = soup.select('table.info td')
#     all_txt = [td.get_text() for td in cells]
#     match  = BLOCK_RE.search(all_txt[1])
#     if match:blockId = int(match.group())
#     else:   blockId =  None
#     match_2 = FEE_RE.search(all_txt[4]) 
#     if match_2: fee = float(match_2.group())  
#     else: fee= None
#     return {
#         "timestamp":all_txt[2],
#         "blockId":blockId,
#         "txId":all_txt[0],
#         "isCoinbase":0,
#         "fee":fee
#     }


# def get_wallet_adress(trans_table_link,session):
#     res = safe_get(session,f'{"https://www.walletexplorer.com"}{trans_table_link}')
#     soup = BeautifulSoup(res.text,"html.parser")
#     cells = soup.select('table.info td')
#     all_txt = [td.get_text() for td in cells]
#     return all_txt[3]

# def get_all_trans(transaction_list):
#     infos = []
#     session = requests.Session()
#     session.headers.update(headers)
#     for transaction_link in transaction_list:
#         # info = get_info_from_transaction_table(transaction_link,session)
#         info = get_wallet_adress(transaction_link,session)
#         infos.append(info)
#     return infos

<center>

# Analisi Transazioni DiceOnCrack


In [136]:
diceoncrack_transaction = set(scrape_wallet_explorer('DiceonCrack'))
print(len(diceoncrack_transaction))

[<a href="/address/12TaAbLWBNKB1NLYH92CPnC1DizQoNK6FN">12TaAbLWBNKB1NLYH92CPnC1DizQoNK6FN</a>, <a href="/address/1CRACkbiJSxfDaLNEoaNsHjNtU4KttwHyo">1CRACkbiJSxfDaLNEoaNsHjNtU4KttwHyo</a>, <a href="/address/1CRACKafkXsQzUYmu2fUM3j9c2y4yDhvfh">1CRACKafkXsQzUYmu2fUM3j9c2y4yDhvfh</a>, <a href="/address/1CRACKLiwFrZbAQz1yb9w22onHCMLbiMTY">1CRACKLiwFrZbAQz1yb9w22onHCMLbiMTY</a>, <a href="/address/12tAabLFLxvUzC5KuX7VKMM8bYRncbQ84E">12tAabLFLxvUzC5KuX7VKMM8bYRncbQ84E</a>, <a href="/address/1CrAcKt3HE8LNsx4KKDvjqLvcr373wg5ke">1CrAcKt3HE8LNsx4KKDvjqLvcr373wg5ke</a>, <a href="/address/1AVFypuG2jUrYzjZa69C7hK59XkWUwvK1m">1AVFypuG2jUrYzjZa69C7hK59XkWUwvK1m</a>, <a href="/address/1CRACK25QvpVdcEmPZVD5ixtf99cMF9stg">1CRACK25QvpVdcEmPZVD5ixtf99cMF9stg</a>, <a href="/address/1CracksLRtQMcTF4HXNrvPzRgvz7Qr6wNd">1CracksLRtQMcTF4HXNrvPzRgvz7Qr6wNd</a>, <a href="/address/13TAabLHjNzwg8Mj7XYn76FuVAqj32s8EM">13TAabLHjNzwg8Mj7XYn76FuVAqj32s8EM</a>, <a href="/address/1CrAckQppdcfiiw4XzpsKrZrf9eDvUok9C">1CrAc

In [137]:
display(diceoncrack_transaction)

{'12TaAbLWBNKB1NLYH92CPnC1DizQoNK6FN',
 '12tAabLFLxvUzC5KuX7VKMM8bYRncbQ84E',
 '13TAabLHjNzwg8Mj7XYn76FuVAqj32s8EM',
 '13encD1Yagh8M6a9Wgb3YJxKHrHqXnYi8y',
 '14TAAbLiw2QLuRJCGQ3iETYg3vcpweZkTE',
 '15TaABLmhxiRQ9DTX6ZcZ9S9RknVZmP5jX',
 '17ZmFwCULT44K25kWDeYbHiGaJCrWtytjx',
 '19TAABLQTLxgWHTdm7yNJNstgeQFgxTP4f',
 '1AVFypuG2jUrYzjZa69C7hK59XkWUwvK1m',
 '1CRACK25QvpVdcEmPZVD5ixtf99cMF9stg',
 '1CRACKLiwFrZbAQz1yb9w22onHCMLbiMTY',
 '1CRACKafkXsQzUYmu2fUM3j9c2y4yDhvfh',
 '1CRACkbiJSxfDaLNEoaNsHjNtU4KttwHyo',
 '1CrAcKt3HE8LNsx4KKDvjqLvcr373wg5ke',
 '1CrAckQppdcfiiw4XzpsKrZrf9eDvUok9C',
 '1CracksLRtQMcTF4HXNrvPzRgvz7Qr6wNd',
 '1F4VXTQRzVQfLaGEWcf697xj1g2cKqPire',
 '1GD2EiVa1rbbXcmFceyM47YN16fzVwn9j',
 '1LQXotaEjfmerkwrGB3dHnheujo7sng6vA',
 '1PipEaL8yRS8n93mUS16wT5SNDiMrMutv5',
 '1PipeBMryPGnN3Ms3HfnNjetCS4THmkpkS',
 '1PipePezjvE7vBukPyDUkhHEF54qK1nkeu',
 '1PipeZHgQXcjAYsUQ4WRXyKZn1X3sJNrpk',
 '1PipeZofhJv1hxsxCadEeG1vHAK87f23LE',
 '1Pipeb5iNYmURifrxPZfvwHsTiw9rEb2iu',
 '1PipemCUjxq9LKww7CaLWUMe

In [139]:
doc_addrs = merged_outputs[merged_outputs["hash"].isin(diceoncrack_transaction)]
print(len(doc_addrs))

17034


### Salvataggio delle transazioni in Csv per velocizzare il testing

In [ ]:
# # salva il DataFrame in un CSV nella cartella corrente
# doc_df = pd.DataFrame(diceoncrack_transaction,columns=["hash"]).set_index("hash")
# doc_df.to_csv("diceoncrack.csv", index=True, encoding="utf-8")

### Recupero dei dati dal csv

In [113]:
dice_df = pd.read_csv('diceoncrack.csv',index_col="hash")
dice_set = set(dice_df.index.to_list())

<center>

# Analisi Transazioni di DeepBit.Net

In [140]:
deepbit_transactions = set(scrape_wallet_explorer('DeepBit.net'))
print(len(deepbit_transactions))

[<a href="/address/1VayNert3x1KzbpzMGt2qdqrAThiRovi8">1VayNert3x1KzbpzMGt2qdqrAThiRovi8</a>, <a href="/address/13NGmRF2SVRg3aKdGNVhXLmhA1JT9p87a8">13NGmRF2SVRg3aKdGNVhXLmhA1JT9p87a8</a>]
2


In [ ]:
print(len(deepbit_transactions))
display(deepbit_transactions)
bnet_addrs = merged_outputs[merged_outputs["hash"].isin(deepbit_transactions)]
print(len(bnet_addrs))

2


{'13NGmRF2SVRg3aKdGNVhXLmhA1JT9p87a8', '1VayNert3x1KzbpzMGt2qdqrAThiRovi8'}

17034


In [ ]:
# # salva il DataFrame in un CSV nella cartella corrente
# doc_df = pd.DataFrame(deepbit_transactions,columns=["inputs/outputs"]).set_index("inputs/outputs")
# doc_df.to_csv("deepbit.csv", index=True, encoding="utf-8")

In [ ]:
# deepbit_df = pd.read_csv('deepbit.csv',index_col="inputs/outputs")

doc_addrs = merged_outputs[merged_outputs["hash"].isin(diceoncrack_transaction)]
print(len(doc_addrs))